# Apriori Algorithm

## preprocessing our file

we first preprocess our retail.txt file to include commas between it using:

1. The function in notepad to replace the space with commas

2. with open(r'C:\Users\Vijay Maruthi\Downloads\retail.txt','r') as file,open(r'C:\Users\Vijay           Maruthi\Downloads\_no_commas.txt','w') as target:
    target.write(file.read().replace(',\n','\n'))
    
The second step is used to remove trailing commas

## The code below is quite slow

In [1]:
import numpy as np
from itertools import combinations, chain
import time

def load_transactions (path_to_data, order):
    Transactions = []
    with open (path_to_data, 'r') as fid:
        for lines in fid:
            str_line = list (lines.strip().split(','))
            _t = list(np.unique(str_line))
            _t.sort(key=lambda x: order.index(x))
            Transactions.append(_t)
    return Transactions

def count_occurences(itemset, Transactions):
    count=0
    for i in range(len(Transactions)):
        if set(itemset).issubset(set(Transactions[i])):
            count+=1
    return count


def join_two_itemsets (it1, it2, order):
    it1.sort(key=lambda x: order.index(x))
    it2.sort(key=lambda x: order.index(x))



    for i in range (len (it1)-1):
        if it1[i] != it2 [i]:
            return []
    
    if order.index(it1[-1]) < order.index(it2[-1]):
        return it1 + [it2[-1]]
    return []

def join_set_itemsets(set_of_its, order):
    C = []
    for i in range (len (set_of_its)):
        for j in range (i+1, len(set_of_its)):
            it_out = join_two_itemsets (set_of_its[i], set_of_its[j], order)
            if len(it_out) > 0:
                C.append(it_out)

    return C

def get_frequent(itemsets, Transactions, min_support, prev_discarded):
    L = []
    supp_count = []
    new_discarded = []
    k = len(prev_discarded.keys())
    
    for s in range (len (itemsets)):
        discarded_before = False
        if k > 0:
            for it in prev_discarded[k]:
                if set(it).issubset (set(itemsets[s])):
                    discarded_before = True
                    break
                    
        if not discarded_before:
            count= count_occurences (itemsets[s], Transactions)
            if count/len(Transactions) >= min_support:
                L.append(itemsets[s])
                supp_count.append(count)

            else:
                new_discarded.append(itemsets[s])
    return L, supp_count, new_discarded

def print_table(T,supp_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
        print("{} | {} ".format(T[k],supp_count[k]))
    print("\n\n")

def powerset(s):
    return list(chain.from_iterable (combinations(s, r) for r in range(1, len(s)+ 1)))

def write_rules(X, X_S, S, conf, supp, lift, num_trans):
    out_rules=""
    out_rules += "Freq. Itemset: \n".format(X)
    out_rules += " Rule: {} -> {} \n".format(list(S), list (X_S))
    out_rules += " Confidence: {0:2.3f} ".format(conf)
    out_rules += ",Support: {0:2.3f} ".format(supp/num_trans)
    out_rules +=" ,Lift: {0:2.3f} \n".format(lift)
    return out_rules

In [2]:
# Loading the data

path_to_data="retail_short.txt" 

min_support=float(input("Enter minimum support value: "))
min_confidence=float(input("Enter Minimum confidence value :"))

order = [str(i) for i in range(20000)]

Transactions=load_transactions(path_to_data, order)
num_trans=len(Transactions)

Enter minimum support value: 0.05
Enter Minimum confidence value :0.2


# Initialization

In [3]:
begin = time.time()

C = {}
L = {}

itemset_size = 1
discarded={itemset_size : []}
C.update({itemset_size: [[f] for f in order]})

## Create L1

In [4]:
supp_count_L={}
f, sup, new_discarded = get_frequent(C[itemset_size],Transactions,min_support,discarded)
discarded.update({itemset_size : new_discarded})
L.update({itemset_size : f})
supp_count_L.update({itemset_size : sup})

In [5]:
print("L1:\n")
print_table(L[1], supp_count_L[1])

L1:

Itemset | Frequency
['32'] | 204 
['38'] | 362 
['39'] | 882 
['41'] | 375 
['48'] | 648 
['1327'] | 76 





In [6]:
# CIK (we are going to need the join step from L[k-1])

k=itemset_size+1
convergence=False
while not convergence:
    C.update({ k: join_set_itemsets (L[k-1], order)})
    print("Table C{} : \n".format(k))
    print_table(C[k],[count_occurences(it, Transactions) for it in C[k]])
    f, sup, new_discarded =get_frequent(C[k], Transactions, min_support, discarded)
    discarded.update({k: new_discarded})
    L.update({k: f})
    supp_count_L.update({k: sup}) 
    if len(L[k]) == 0:
        convergence =True
    else:
        print("Table L{} \n".format(k))
        print_table(L[k],supp_count_L[k])
    k+=1

Table C2 : 

Itemset | Frequency
['32', '38'] | 59 
['32', '39'] | 116 
['32', '41'] | 59 
['32', '48'] | 100 
['32', '1327'] | 10 
['38', '39'] | 234 
['38', '41'] | 133 
['38', '48'] | 162 
['38', '1327'] | 17 
['39', '41'] | 292 
['39', '48'] | 468 
['39', '1327'] | 51 
['41', '48'] | 196 
['41', '1327'] | 22 
['48', '1327'] | 38 



Table L2 

Itemset | Frequency
['32', '39'] | 116 
['32', '48'] | 100 
['38', '39'] | 234 
['38', '41'] | 133 
['38', '48'] | 162 
['39', '41'] | 292 
['39', '48'] | 468 
['41', '48'] | 196 



Table C3 : 

Itemset | Frequency
['32', '39', '48'] | 67 
['38', '39', '41'] | 105 
['38', '39', '48'] | 125 
['38', '41', '48'] | 70 
['39', '41', '48'] | 161 



Table L3 

Itemset | Frequency
['38', '39', '41'] | 105 
['38', '39', '48'] | 125 
['39', '41', '48'] | 161 



Table C4 : 

Itemset | Frequency
['38', '39', '41', '48'] | 59 





## Generating the association rules

In [7]:


assoc_rules_str =""

for i in range(1, len(L)): 
    for j in range(len (L[i])):
        s = powerset(L[i][j])
        s.pop()
        for z in s:
            S = set(z)
            X= set (L[i] [j])
            X_S = set(X-S)
            sup_x= count_occurences (X, Transactions)
            sup_x_s= count_occurences (X_S, Transactions)
            conf = sup_x / count_occurences (S, Transactions)
            lift = conf/ (sup_x_s/ num_trans)
            if conf >= min_confidence and sup_x >= min_support:
                assoc_rules_str+=write_rules (X, X_S, S, conf, sup_x, lift, num_trans)
                
# printing association rules
                
end=time.time()

print("Time taken to generate association rules is :",end-begin)
print()
print(assoc_rules_str)

Time taken to generate association rules is : 25.64456272125244

Freq. Itemset: 
 Rule: ['32'] -> ['39'] 
 Confidence: 0.569 ,Support: 0.077  ,Lift: 0.969 
Freq. Itemset: 
 Rule: ['32'] -> ['48'] 
 Confidence: 0.490 ,Support: 0.067  ,Lift: 1.137 
Freq. Itemset: 
 Rule: ['38'] -> ['39'] 
 Confidence: 0.646 ,Support: 0.156  ,Lift: 1.102 
Freq. Itemset: 
 Rule: ['39'] -> ['38'] 
 Confidence: 0.265 ,Support: 0.156  ,Lift: 1.102 
Freq. Itemset: 
 Rule: ['38'] -> ['41'] 
 Confidence: 0.367 ,Support: 0.088  ,Lift: 1.473 
Freq. Itemset: 
 Rule: ['41'] -> ['38'] 
 Confidence: 0.355 ,Support: 0.088  ,Lift: 1.473 
Freq. Itemset: 
 Rule: ['38'] -> ['48'] 
 Confidence: 0.448 ,Support: 0.108  ,Lift: 1.038 
Freq. Itemset: 
 Rule: ['48'] -> ['38'] 
 Confidence: 0.250 ,Support: 0.108  ,Lift: 1.038 
Freq. Itemset: 
 Rule: ['39'] -> ['41'] 
 Confidence: 0.331 ,Support: 0.194  ,Lift: 1.327 
Freq. Itemset: 
 Rule: ['41'] -> ['39'] 
 Confidence: 0.779 ,Support: 0.194  ,Lift: 1.327 
Freq. Itemset: 
 Rule: ['